<a href="https://colab.research.google.com/github/onleey/chatgpt_demo/blob/master/007_LangchainChatbot_FAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##여러문서에서 찾아서 답변하는 챗봇 만들기

-랭체인(langchain)이란? ChatGPT와 같은 대규모 언어 모델을 사용하여
애플리케이션을 쉽게 생성할 수 있도록 설계된 프레임 워크이다.  
-랭체인이 도움이 되는 경우는 LLM에 외부의 '지식'이나 '계산능력'을 활용하게 하고 싶을 때이다.  
-자신이 학습만으로 대화하던 LLM에게 '책'이나 '프로그램'을 전달해서 외부의 '지식'이나 '계산능력'을 활용할 수 있게 하는 것이 랭체인의 역할이다.   
-예를들어, 랭체인으로 LLM에 웹 검색 기능을 연결하면 LLM은 자신이 가진 지식만으로는 충분한 답변을 할 수 없는 경우에 웹 검색을 통해 최신 정보를 얻고 답변 할 수 있게 된다.   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/001'

Mounted at /content/drive
/content/drive/MyDrive/001


In [ ]:
!pip install langchain openai==0.28.1 tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

LOAD

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter
from openai.embeddings_utils import get_embedding

In [ ]:
loader = DirectoryLoader('./H_data' , glob='*.txt', loader_cls=TextLoader)

documents = loader.load()

print('문서의 개수:', len(documents))

문서의 개수: 30


In [ ]:
import openai
openai.api_key='api_key'

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'api_key'

In [ ]:
print('1번 문서:', documents[1])
print('-'*20)
print('21번 문서:', documents[2])
print('-'*20)

print( type(documents[1]))
print(dir(documents[1]))
print('-'*20)
documents[1]

1번 문서: page_content='기온이 내려가고 눈이 내린 산이 많아지면서 겨울 등산을 고려하는 이들이 적지 않다. 겨울철 산행은 눈으로 인해 길은 미끄럽고 기온도 낮아 부상 위험이 높다.\n\n\n\n가장 흔한 ‘발목 염좌’, 방치하면 관절염 될 수 있어\n\n\n\n발목염좌는 등산에서 흔한 부상 중 하나다. 울퉁불퉁한 바닥을 걸을 때 헛디뎌 발을 접질리면서 발목의 정상적인 운동 범위를 벗어나면 발목 관절을 잡아주는 인대들이 손상되면서 발생한다. 시간이 지나면 통증이 가라앉기 때문에 치료하지 않고 지나가는 경우가 많은데, 이에 정덕환 교수는 “적절한 시기에 치료하지 않으면 급성 염좌의 약 20% 정도에서 만성 염좌 및 발목 관절 불안정성이 발생할 수 있고, 불안정성이 지속할 경우 장기적으로는 발목 관절염으로 진행할 수 있다”라고 설명했다.\n\n\n\n스틱 등으로 무게 분산, 발목 테이핑 등 도움\n\n\n\n발목 염좌를 예방하려면 입산 전 충분한 스트레칭을 통해 관절 유연성을 높여야 한다. 발에 가해지는 부담은 몸무게에 비례한다. 한 걸음마다 체중의 1.5배, 뛸 때는 4배, 점프할 때는 5배가량의 하중을 견뎌야 한다. 그러므로 너무 무거운 짐을 메고 등산하는 것은 피하고, 등산스틱을 이용해 체중을 발 이외의 곳으로 분산시키는 것도 좋다. 발목에 테이핑하는 것도 발목 보호에 좋은 방법이다.\n\n\n\n미끄러져 낙상 골절, 오랜 등산으로 피로 골절 주의\n\n\n\n겨울철에는 눈이 오거나 서리가 있어 등산길이 더 미끄러운 경우가 많다. 특히 균형감각이 떨어지고 뼈가 약한 노인들은 낙상 시 골절 위험이 더 높아 주의해야 한다. 이외에도 오랜 시간 등산을 하다 보면 뼈에도 스트레스가 쌓이며 피로골절이 발생할 수도 있다.\n\n\n\n아이젠 함께 활용하고 갑자기 무리한 등산 말아야\n\n\n\n낙상 예방을 위해서는 잘 미끄러지지 않고 발목 위까지 올라오는 등산화를 신고 아이젠을 함께 활용하는 것이 좋다. 피로골절은 뼈에 반복적인 하중이 가해지는 일이 갑자기

Document(page_content='기온이 내려가고 눈이 내린 산이 많아지면서 겨울 등산을 고려하는 이들이 적지 않다. 겨울철 산행은 눈으로 인해 길은 미끄럽고 기온도 낮아 부상 위험이 높다.\n\n\n\n가장 흔한 ‘발목 염좌’, 방치하면 관절염 될 수 있어\n\n\n\n발목염좌는 등산에서 흔한 부상 중 하나다. 울퉁불퉁한 바닥을 걸을 때 헛디뎌 발을 접질리면서 발목의 정상적인 운동 범위를 벗어나면 발목 관절을 잡아주는 인대들이 손상되면서 발생한다. 시간이 지나면 통증이 가라앉기 때문에 치료하지 않고 지나가는 경우가 많은데, 이에 정덕환 교수는 “적절한 시기에 치료하지 않으면 급성 염좌의 약 20% 정도에서 만성 염좌 및 발목 관절 불안정성이 발생할 수 있고, 불안정성이 지속할 경우 장기적으로는 발목 관절염으로 진행할 수 있다”라고 설명했다.\n\n\n\n스틱 등으로 무게 분산, 발목 테이핑 등 도움\n\n\n\n발목 염좌를 예방하려면 입산 전 충분한 스트레칭을 통해 관절 유연성을 높여야 한다. 발에 가해지는 부담은 몸무게에 비례한다. 한 걸음마다 체중의 1.5배, 뛸 때는 4배, 점프할 때는 5배가량의 하중을 견뎌야 한다. 그러므로 너무 무거운 짐을 메고 등산하는 것은 피하고, 등산스틱을 이용해 체중을 발 이외의 곳으로 분산시키는 것도 좋다. 발목에 테이핑하는 것도 발목 보호에 좋은 방법이다.\n\n\n\n미끄러져 낙상 골절, 오랜 등산으로 피로 골절 주의\n\n\n\n겨울철에는 눈이 오거나 서리가 있어 등산길이 더 미끄러운 경우가 많다. 특히 균형감각이 떨어지고 뼈가 약한 노인들은 낙상 시 골절 위험이 더 높아 주의해야 한다. 이외에도 오랜 시간 등산을 하다 보면 뼈에도 스트레스가 쌓이며 피로골절이 발생할 수도 있다.\n\n\n\n아이젠 함께 활용하고 갑자기 무리한 등산 말아야\n\n\n\n낙상 예방을 위해서는 잘 미끄러지지 않고 발목 위까지 올라오는 등산화를 신고 아이젠을 함께 활용하는 것이 좋다. 피로골절은 뼈에 반복적인 하중이 가해지는 일이 갑

##split text (텍스트 분할하기)



- ChatGPT API의 GPT-3.5-turbo 모델은 입력으로 사용하는 텍스트와 ChatGPT API가 반환하는 답변의 길이를 모두 포함하여 최대 4,096 토큰을 처리할 수 있다.  
- 예를 들어, ChatGPT API 입력으로 약 3,000토큰 정도의 길이를 가진 입력을 사용하면 ChatGPT는 최대 약 1,000토큰의 답변을 할 수 있는 구조이다.  
- 입력의 길이가 4,096의 길이에 인접한 입력을 사용하면 ChatGPT API가 답변을 하던 도중 4,096개의 토큰을 모두 소진하면 더 이상 답변을 생성할 수 없어 텍스트가 중간에 끊길 수 있다. 또한 입력의 길이가 4,096을 넘으면 ChatGPT의 입력으로 사용할 수 없다는 에러가 발생한다.  

- 이런 현상 때문에 너무 긴 문서들은 한 번에 입력으로 사용할 수 없으므로 여러 개의 뭉치(chunk)로 자른 후 ChatGPT API를 여러 번 호출하여 처리해야 한다.  
- 랭체인의 RecursiveCharacterTextSplitter는 이를 위해 사용하는 도구로, 주어진 문서를 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 한다.  
-chunk_size에는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap에는 텍스트를 분할할 때 텍스트의 내용을 얼마나 겹쳐서 자를 것인지 지정한다.  

In [ ]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수:', len(texts))

분할된 텍스트의 개수: 93


In [ ]:
texts[0]

Document(page_content="대부분 무증상 초음파?세포검사 통해 진단…변형 일어나기도 주기적 추적관찰 필요\n\n\n고대구로병원 송의연 교수 “크기 큰 양성 결절로 목소리 변화 시 수술 고려해야”\n\n\n[의학신문·일간보사=정광성 기자] 목의 한가운데에서 앞으로 튀어나온 물렁뼈(갑상연골, 甲狀軟骨)의 아래쪽 기도의 주위를 감싸고 있는 내분비선인 ‘갑상선’은 음식물을 통해 섭취한 아이오딘을 이용해 갑상선 호르몬을 생성한 후 체내로 분비해 우리몸의 대사와 체온조절 등 인체 내 모든 기관의 기능을 적절하게 유지시킨다.\n\n\n\n이러한 갑상선에 세포의 과증식으로 발생하는 ‘갑상선 결절’은 갑상선 질환 중 가장 흔한 질환이다. 건강보험심사평가원에 따르면 2017년 27만4799명에서 2022년 40만9609명으로 5년간 49% 증가세를 보이고 있다.\n\n\n\n이렇게 흔하게 생길 수 있는 갑상선 결절은 무조건적으로 치료하는 것은 아니다. 갑상선상의 결절이 보일 경우 미세침흡인 세포검사를 통해 양성/악성여부를 확인한다.\n\n\n\n갑상선 결절은 대부분 양성결절이나 악성일 경우 전문의와의 진료를 통해 적극적으로 치료를 해야 한다.\n\n\n\n◇특별한 증상 없는 ‘갑상선 결절', 대부분 검진을 통해 발견=갑상선 결절은 정상적인 세포가 과도하게 증식해서 발생하는데 아직까지 증식의 원인은 명확하게 밝혀지지 않았다. 다양한 환경적 및 유전적 요소가 복합적으로 작용할 것으로 추측되고 있다.\n\n\n\n갑상선 결절은 대부분 특별한 증상이 없고, 초음파 검사로 우연히 발견되는 경우가 가장 흔하다. 만약 결절의 크기가 수주 또는 수개월 사이에 갑자기 커졌거나, 목이 쉬는 등 목소리의 변화가 발생한 경우, 호흡 곤란, 삼킴 곤란, 통증이 발생한 경우에는 갑상선 암일 경우가 높으므로 반드시 전문의의 진료를 보는 것이 좋다.", metadata={'source': 'H_data/1.txt'})

In [ ]:
source_lst=[]
for i in range(0,len(texts)):
  source_lst.append(texts[i].metadata['source'])

element_counts =Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >=2}
print('2개 이상으로 분할된 문서 :',filtered_counts)
print('분할된 텍스트의 개수:',len(documents)+len(filtered_counts))

2개 이상으로 분할된 문서 : {'H_data/1.txt': 3, 'H_data/2.txt': 4, 'H_data/3.txt': 2, 'H_data/4.txt': 3, 'H_data/5.txt': 4, 'H_data/6.txt': 3, 'H_data/7.txt': 4, 'H_data/8.txt': 2, 'H_data/9.txt': 2, 'H_data/10.txt': 3, 'H_data/12.txt': 4, 'H_data/13.txt': 3, 'H_data/14.txt': 4, 'H_data/15.txt': 2, 'H_data/16.txt': 3, 'H_data/17.txt': 3, 'H_data/18.txt': 2, 'H_data/19.txt': 4, 'H_data/20.txt': 3, 'H_data/21.txt': 2, 'H_data/22.txt': 4, 'H_data/23.txt': 2, 'H_data/24.txt': 3, 'H_data/25.txt': 2, 'H_data/26.txt': 5, 'H_data/27.txt': 4, 'H_data/28.txt': 5, 'H_data/29.txt': 2, 'H_data/30.txt': 5}
분할된 텍스트의 개수: 59


##Create Chroma DB

get_embedding('저는 배가 고파요', engine='text-embedding-ada-002')

def cos_sim(A,b):
  return dot(A,B)/ (norm(A) * norm(B))


  -ChormDB는 이러한 과정들을 기능별로 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있게 도화주는 편리한 벡터응용 도구이다.  
  -ChormDB.from_documents( ) 함수를 이용해 벡터도구 객체를 선언한다. 이때 documents에서는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 전달하고, embedding에는 어떤 종류의 임베딩을 사용 할 것인지를 기재한다.

그동안에는 OPEN AI Embedding API를 이용해 텍스트를 임베딩하고, 코사인 유사도를 계산해 유사한 텍스트를 가져오는 작업을 했다.

In [ ]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

print(vectordb)
print(dir(vectordb))

['_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_client', '_client_settings', '_collection', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_persist_directory', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_images', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_sea

## Make a retriever (검색기)
- 벡터 도구 객체를 선언하고 나면 as_retriever()함수를 이용해 입력된 텍스트로부터 유사한 텍스트를 찾아주는 검색기 (retriever)를 선언한다.


In [ ]:
retriever = vectordb.as_retriever()
print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [ ]:
docs = retriever.get_relevant_documents('당뇨가뭐야?')
print('유사문서 개수:', len(docs))
print('--'*20)
print('첫번째 유사문서:', docs[0])
print('--'*20)
print('각 유사 문서의 문서 출처:')

for doc in docs:
     print(doc.metadata['source'])



유사문서 개수: 4
----------------------------------------
첫번째 유사문서: page_content='당뇨 환자, 건강한 설 보내기\n\n\n당뇨병을 앓고 있는 환자에게 명절은 이래저래 고민거리를 가져다 주기도 하는데, 실제 명절 후에 평소보다 혈당이 높아지는 경향을 보이기 때문이다. 명절이면 자고로 가족과 함께 정성을 다해 만드는 음식이 떠오르는데, 평소 먹기 쉽지 않고 맛이 좋아 자꾸 손이 가게 된다. 하지만 영양학적으로는 열량을 많이 함유하고 있을 뿐만 아니라 기름져 자칫 혈당이나 체중 조절에 어려움을 겪게 한다. 그렇다면 당뇨병 환자가 명절을 슬기롭게 보내려면 어떤 걸 주의해야 할까?\n\n우선, 과식하지 않는 것이 가장 중요하다. 명절에는 여러 종류의 음식이 있어 조금씩만 먹더라도 과식하기 쉬우며, 대부분 열량이 높고 기름진 음식이라 혈당 및 체중 조절이 어렵다. 최대한 먹더라도 평소 포만감을 느낄 정도로만 먹는 것이 좋으며, 얼핏 보았을 때 달고 기름진 것은 피해야 한다.\n\n또한, 먹은 양과 운동량의 균형을 맞추어야 한다. 계속 망설이다가 하는 군것질 후에는 혈당을 떨어트리기 위해 보통 생각하는 것보다 운동을 훨씬 많이 해야 한다. 평소 정해진 식단 이외 추가로 섭취한 음식이나 간식의 열량을 알아본 후, 섭취 열량과 운동량의 균형을 따져보아야 한다.\n\n도로 사정이 좋지 않은 명절에는 당뇨병 약을 복용 중인 환자의 경우 정해진 식사 시간을 놓친다면 저혈당에 빠질 위험이 있다. 장거리 운전을 한다면 저혈당 대비용 사탕을 가지고 다녀야 하며, 무엇보다 안전수칙에 맞추어 쉬엄쉬엄 운전하는 것이 좋다.\n\n평소 바쁘다는 이유로 지키지 못했던 당뇨병 환자는 스스로 자신의 모습을 다시 되짚어보는 것이 좋다. 긴 명절 연휴에 생활수칙을 잘 지킬 수 있을지, 아니면 평소에도 충실한 생활로 잘 지킬 수 있었던 수칙은 아니었는지 스스로 판단해본다.' metadata={'source': 'H_data/25.txt'}
------

In [ ]:
#search_kwargs={'k':문서의 개수}를 선언하면 유사도 문저갯수가 지정한 갯수만큼 리턴해준다.
retriever = vectordb.as_retriever(search_kwargs={'k':2})


In [ ]:
docs = retriever.get_relevant_documents('갑상선에 도움이되는 음식이 있어?')
for doc in docs:
     print(doc.metadata['source'])

H_data/26.txt
H_data/25.txt


## MAKE a Chain

```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

In [ ]:
#RetrievalQA.from_chain_type( )함수의 llm 매개변수 값으로 ChatOpenAI( )를 지정하고,
#내부매개변수로 model_name의 값으로 'gpt-3.5-turbo'를 지정하면 ChatGPT API를 사용할 수 있다.
#temperature=0이면 같은 답변을 반환하고, 1이면 랜덤이 발생한다. 기본값은 1이다.
#chain type= stuff 이면 내부적으로 위에있는 문장처럼 수행한다.
#{텍스트}에는 입력데이터와 유사도가 높은 텍스트의 본문이 삽입되고, {질문}에는 입력텍스트가 삽입되는구조이다.
#그동안 사용자가 직접 프롬프트를 작성한 것과 달리 이미 여기서는 이미 작성된 프롬프트를 사용한다.

qa_chain = RetrievalQA.from_chain_type (
    llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
    chain_type = 'stuff',
    retriever=retriever,
    return_source_documents=True)


Query

In [ ]:
input_text = '당뇨와 관련된 질환이 궁금합니다'
chatbot_response = qa_chain(input_text)
print(chatbot_response)
print(dir(chatbot_response))
print(chatbot_response.keys)

{'query': '당뇨와 관련된 질환이 궁금합니다', 'result': '당뇨와 관련된 질환이 여러 가지 있을 수 있습니다. 일반적으로 당뇨병은 혈당 조절에 문제가 있는 질환으로, 만성적인 고혈당 상태가 지속되면 다양한 합병증이 발생할 수 있습니다. \n\n가장 흔한 당뇨 합병증은 신장 질환인 당뇨 신병입니다. 고혈당이 지속되면 신장의 혈관이 손상되어 신장 기능이 저하되고, 신장 손상에 따른 당뇨 신병이 발생할 수 있습니다. 이는 신장 기능 저하, 단백뇨, 고혈압 등으로 나타날 수 있습니다.\n\n또한, 당뇨로 인해 혈관 손상이 발생할 수 있으며, 이는 말초혈관 질환인 당뇨병 말초혈관병증을 일으킬 수 있습니다. 이는 손발의 혈액 공급이 저하되어 근육 통증, 근력 감퇴, 궤양, 궤양이 감염되어 근육을 파괴하는 등의 증상을 유발할 수 있습니다.\n\n또한, 당뇨로 인해 신경 손상이 발생할 수 있으며, 이는 당뇨 신경병증으로 알려져 있습니다. 이는 신경 손상으로 인해 손과 발의 감각 이상, 통증, 근육 약화 등의 증상을 유발할 수 있습니다.\n\n이 외에도 당뇨로 인해 발생할 수 있는 합병증으로는 당뇨망막병증, 당뇨 심장병, 당뇨 신경병증, 당뇨 발병증 등이 있습니다. 이러한 합병증은 당뇨의 혈당 조절을 철저히 관리하고, 건강한 생활습관을 유지하는 것이 중요합니다. 당뇨와 관련된 질환에 대해서는 당뇨 전문의와 상담하여 자세한 정보를 얻는 것이 좋습니다.', 'source_documents': [Document(page_content='피부질환이 동반되지 않은 경우 혈액검사를 통해 신장·간·갑상선 및 빈혈 수치 등을 확인하고 소변검사를 통한 당뇨여부, 흉부 X선 검사, 간염 및 매독, HIV(인간면역결핍바이러스) 항체 검사 등을 시행한다. 또한 필요시 악성 질환 감별을 위한 선별 검사를 진행한다. 이외에 신경학적, 정신적 요인을 확인하는 것도 중요하다.\n\n\n\n치료는 가려움증의 원인 제거 및 증상 완화를 위한 치료를 병행하는 것이 중요하다. 항히스타민제는 가려

In [ ]:
print(chatbot_response.keys())

dict_keys(['query', 'result', 'source_documents'])


In [ ]:
def get_chatbot_response(chatbot_response):
  print(chatbot_response['result'].strip())
  print('\n문서출처:')
  for source in chatbot_response['source_documents']:
     print(source.metadata['source'])

In [ ]:
input_text = '당뇨를 예방할 방법이 있을까?'
chatbot_response= qa_chain(input_text)
get_chatbot_response(chatbot_response)

당뇨병을 예방하기 위해서는 다음과 같은 방법들을 실천할 수 있습니다:

1. 건강한 식습관 유지: 균형 잡힌 식단을 유지하고, 과다한 탄수화물과 당의 섭취를 제한하는 것이 중요합니다. 신선한 과일과 채소, 고기와 양파 등의 단백질을 포함한 식품을 섭취하는 것이 좋습니다.

2. 체중 관리: 비만은 당뇨병 발병 위험을 증가시키므로, 건강한 체중을 유지하는 것이 중요합니다. 규칙적인 운동과 적절한 식단 조절을 통해 체중을 관리할 수 있습니다.

3. 규칙적인 운동: 일주일에 최소 150분 이상의 유산소 운동을 실천하는 것이 좋습니다. 운동은 혈당 조절에 도움을 주고, 체중 감량과 근육 강화에도 도움이 됩니다.

4. 스트레스 관리: 스트레스는 혈당 조절에 영향을 줄 수 있으므로, 스트레스를 관리하는 방법을 찾는 것이 중요합니다. 명상, 요가, 걷기 등의 활동을 통해 스트레스를 해소할 수 있습니다.

5. 정기적인 건강 검진: 당뇨병의 초기 증상을 빠르게 발견하고 예방하기 위해 정기적인 건강 검진을 받는 것이 중요합니다. 혈당 수치와 다른 건강 지표를 모니터링하여 조기에 대처할 수 있습니다.

이러한 예방 방법들을 실천하면 당뇨병 발병 위험을 줄일 수 있습니다. 하지만 개인의 상황에 따라 적절한 조언을 받기 위해 의사나 건강 전문가와 상담하는 것이 좋습니다.

문서출처:
H_data/28.txt
H_data/25.txt


In [ ]:
input_text = '가려움증 원인이 뭐야?'
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

가려움증의 원인은 다양할 수 있습니다. 일반적으로 피부질환, 전신 질환, 신경학적 원인, 정신적 원인 등이 가려움증을 유발할 수 있습니다. 피부질환으로는 피부건조증, 아토피피부염, 건선, 두드러기, 접촉 피부염, 편평태선, 결절성 양진, 옴, 곤충 물림, 무좀 등이 있습니다. 전신 질환으로는 만성 신장질환, 만성 간질환, 담즙 정체, 당뇨병, 갑상선 기능 항진 및 저하증, 고형암, 백혈병, 림프종, 진성적혈구증가증, 빈혈, 후천성면역결핍증 등이 있습니다. 신경학적 원인으로는 다발성 경화증, 상완요골 가려움증, 이상감각등신경통, 대상포진 등이 있습니다. 정신적 원인으로는 강박반응성 장애 등이 있습니다. 하지만 가려움증의 원인은 개인마다 다를 수 있으므로, 자세한 병력 청취와 진찰을 통해 원인을 찾는 것이 중요합니다.

문서출처:
H_data/30.txt
H_data/30.txt


In [ ]:
query= "가려움증을 예방하려면?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

가려움증을 예방하기 위해서는 다음과 같은 조치를 취할 수 있습니다:

1. 피부를 적절하게 보호하세요: 피부를 건강하게 유지하기 위해 적절한 보습제를 사용하고, 피부를 건조하게 만드는 요인을 피하세요. 또한 피부를 자극하는 화학물질이나 알레르기 원인물질을 피하도록 주의하세요.

2. 스트레스를 관리하세요: 스트레스는 가려움증을 악화시킬 수 있는 요인입니다. 스트레스 관리 기술을 사용하여 스트레스를 줄이고, 신체와 마음의 안정을 유지하세요.

3. 옷차림과 환경을 조절하세요: 피부를 자극하는 옷이나 환경 요인을 피하고, 피부를 보호할 수 있는 옷을 착용하세요. 또한 건조한 환경에서는 가습기를 사용하여 공기를 적절하게 유지하세요.

4. 올바른 식습관을 유지하세요: 올바른 식습관은 피부 건강에도 중요한 역할을 합니다. 식이 섬유를 충분히 섭취하고, 수분을 충분히 섭취하여 피부를 건강하게 유지하세요.

5. 알레르기 원인물질을 피하세요: 알레르기 반응을 유발하는 원인물질을 피하고, 알레르기 테스트를 받아 알레르기를 관리하세요.

6. 피부 관리를 철저히 하세요: 피부를 깨끗하게 유지하고, 적절한 피부 관리 제품을 사용하여 피부를 보호하세요. 피부에 자극을 주는 화학물질이나 알레르기 원인물질을 피하도록 주의하세요.

7. 의사와 상담하세요: 만성 가려움증이나 피부 질환의 경우, 피부과 전문의와 상담하여 개별화된 치료 방법을 받으세요. 의사의 지도를 따르면 가려움증을 예방하고 관리할 수 있습니다.

문서출처:
H_data/30.txt
H_data/30.txt


In [ ]:
query= "불면증에대해서 알려줘"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

불면증은 잠들기 어렵거나, 잠이 들어도 자주 깨거나, 새벽에 일찍 깨서 잠이 안 오는 증상이 복합적으로 나타나는 질환이다. 불면증은 누구나 한 번쯤 겪을 수 있지만, 주 3회 이상, 3개월 이상 지속되면 만성 불면증으로 분류된다.

급성 불면증은 다음날 중요한 시험 등 신경 쓰이는 일이 있거나 스트레스, 우울, 불안, 통증, 카페인, 술, 질병, 환경 등이 주요 원인이며, 유발 요인 없이 나타나기도 한다. 원인을 제거하면 자연스럽게 호전될 수 있다. 수면 위생을 잘 지키고 불면증을 악화시키는 심리적, 인지적, 행동적 요인들을 중재하는 인지행동치료가 도움이 될 수 있다.

만성 불면증의 경우 불면증 외에도 다른 수면질환이 동반됐거나, 다른 수면질환이 불면증처럼 보이는 것일 수도 있다. 이러한 질환들은 단순 불면증과는 치료 방법이 다르므로 잘 감별해 함께 치료하는 것이 중요하다.

불면증은 누구나 한 번쯤 겪을 수 있지만, 만성 불면증은 주의가 필요하며, 적절한 치료를 받아야 한다. 만성 불면증의 경우 수면 병력 확인, 수면 일기 작성, 수면각성활동량검사 등을 통해 정확한 진단을 받고 적절한 치료 방법을 선택해야 한다.

문서출처:
H_data/7.txt
H_data/7.txt


In [ ]:
query= "생리통이 너무 심해"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

생리통이 심한 경우, 자궁내막종을 의심해볼 수 있습니다. 자궁내막종은 극심한 월경통을 유발할 수 있으며, 난임과 관련된 문제도 초래할 수 있습니다. 자궁내막종은 건강검진을 통해 발견되는 경우가 많으며, 조기 치료가 필요합니다. 자세한 진단과 치료는 의사와 상담하여 결정해야 합니다.

문서출처:
H_data/18.txt
H_data/5.txt


In [ ]:
print(chatbot_response)
print('-------------')
chatbot_response

{'query': '당뇨를 예방할 방법이 있을까?', 'result': '당뇨병을 예방하기 위해서는 다음과 같은 방법들을 실천할 수 있습니다:\n\n1. 건강한 식습관 유지: 균형 잡힌 식단을 유지하고, 과다한 탄수화물과 당의 섭취를 제한하는 것이 중요합니다. 신선한 과일과 채소, 고기와 양파 등의 단백질을 포함한 식품을 섭취하는 것이 좋습니다.\n\n2. 체중 관리: 비만은 당뇨병 발병 위험을 증가시키므로, 건강한 체중을 유지하는 것이 중요합니다. 규칙적인 운동과 적절한 식단 조절을 통해 체중을 관리할 수 있습니다.\n\n3. 규칙적인 운동: 일주일에 최소 150분 이상의 유산소 운동을 실천하는 것이 좋습니다. 운동은 혈당 조절에 도움을 주고, 체중 감량과 근육 강화에도 도움이 됩니다.\n\n4. 스트레스 관리: 스트레스는 혈당 조절에 영향을 줄 수 있으므로, 스트레스를 관리하는 방법을 찾는 것이 중요합니다. 명상, 요가, 걷기 등의 활동을 통해 스트레스를 해소할 수 있습니다.\n\n5. 정기적인 건강 검진: 당뇨병의 초기 증상을 빠르게 발견하고 예방하기 위해 정기적인 건강 검진을 받는 것이 중요합니다. 혈당 수치와 다른 건강 지표를 모니터링하여 조기에 대처할 수 있습니다.\n\n이러한 예방 방법들을 실천하면 당뇨병 발병 위험을 줄일 수 있습니다. 하지만 개인의 상황에 따라 적절한 조언을 받기 위해 의사나 건강 전문가와 상담하는 것이 좋습니다.', 'source_documents': [Document(page_content='먼저 정기검진에서 당뇨병 전단계라는 진단을 받게 되면 규칙적인 식습관, 균형적인 식단과 함께 운동 시작을 권한다. 비만이라면 체중감량을 하는 것도 도움이 된다. 무엇보다 1년에 1회 정기적인 검사는 필수다. 당뇨병 고위험군의 경우 운동, 식이조절을 통해 체중 조절 시 당뇨병 발생을 예방함은 물론 효과가 10여 년 이상 지속된다는 연구 결과도 있다.\n\n\n\n김은숙 교수는 “우리가 안경을 쓰는 것을 완전히 치료했다고 하지 않듯 당뇨병도

{'query': '당뇨를 예방할 방법이 있을까?',
 'result': '당뇨병을 예방하기 위해서는 다음과 같은 방법들을 실천할 수 있습니다:\n\n1. 건강한 식습관 유지: 균형 잡힌 식단을 유지하고, 과다한 탄수화물과 당의 섭취를 제한하는 것이 중요합니다. 신선한 과일과 채소, 고기와 양파 등의 단백질을 포함한 식품을 섭취하는 것이 좋습니다.\n\n2. 체중 관리: 비만은 당뇨병 발병 위험을 증가시키므로, 건강한 체중을 유지하는 것이 중요합니다. 규칙적인 운동과 적절한 식단 조절을 통해 체중을 관리할 수 있습니다.\n\n3. 규칙적인 운동: 일주일에 최소 150분 이상의 유산소 운동을 실천하는 것이 좋습니다. 운동은 혈당 조절에 도움을 주고, 체중 감량과 근육 강화에도 도움이 됩니다.\n\n4. 스트레스 관리: 스트레스는 혈당 조절에 영향을 줄 수 있으므로, 스트레스를 관리하는 방법을 찾는 것이 중요합니다. 명상, 요가, 걷기 등의 활동을 통해 스트레스를 해소할 수 있습니다.\n\n5. 정기적인 건강 검진: 당뇨병의 초기 증상을 빠르게 발견하고 예방하기 위해 정기적인 건강 검진을 받는 것이 중요합니다. 혈당 수치와 다른 건강 지표를 모니터링하여 조기에 대처할 수 있습니다.\n\n이러한 예방 방법들을 실천하면 당뇨병 발병 위험을 줄일 수 있습니다. 하지만 개인의 상황에 따라 적절한 조언을 받기 위해 의사나 건강 전문가와 상담하는 것이 좋습니다.',
 'source_documents': [Document(page_content='먼저 정기검진에서 당뇨병 전단계라는 진단을 받게 되면 규칙적인 식습관, 균형적인 식단과 함께 운동 시작을 권한다. 비만이라면 체중감량을 하는 것도 도움이 된다. 무엇보다 1년에 1회 정기적인 검사는 필수다. 당뇨병 고위험군의 경우 운동, 식이조절을 통해 체중 조절 시 당뇨병 발생을 예방함은 물론 효과가 10여 년 이상 지속된다는 연구 결과도 있다.\n\n\n\n김은숙 교수는 “우리가 안경을 쓰는 것을 완전히 치료했다고 하지 않듯 당뇨

### Gradio 챗봇의 UI 만들기


In [ ]:
!pip install gradio==3.40.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 53.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.9 MB/s eta 0:00:0

In [ ]:
import gradio as gr

#인터페이스를 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label = '건강질문챗봇')
    msg = gr.Textbox(label = '질문해주세요!')
    clear = gr.Button('대화 초기화') #대화초기화 버튼

     #챗봇의 답변을 처리하는 함수

    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += '#sources :'

      #답변 출처
      for i, doc in enumerate(result['source_documents']):
         bot_message += '['+ str(i+1) +']' + doc.metadata['source']+ ' '

        #채팅기록에 사용자의 메시지와 봇의 응답을 추가
      chat_history.append((message,bot_message))
      return"", chat_history

        #사용자의 입력을 제출하면 respond함수가 호출
    msg.submit(respond, [msg,chatbot], [msg,chatbot])

        #초기화 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda:None,None, chatbot, queue=False)

#인터페이스 실행
demo.launch(debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
